In [6]:
import fileinput
import string
from collections import defaultdict, Counter
from itertools import filterfalse
import random

https://www.nytimes.com/games/wordle/index.html

In [26]:
def inc_exc(words, include, exclude):
    include, exclude = set(include), set(exclude)
    possible = set(words)
    
    for ch in include:
        possible &= letters[ch]

    print(f'Words that include {include}: {len(possible):,}')
    for ch in exclude:
        possible -= letters[ch]

    print(f'Words that include {include}, but exclude {exclude}: {len(possible):,}')
    
    return possible

In [2]:
words = [word.strip() for word in fileinput.input('words_alpha.txt')]
print(f'All words: {len(words):,}')

All words: 370,105


## Guesses

### Method 1: Uniformally random word

In [3]:
distinct_letters = list(filter(lambda word: len(set(word)) == len(word) == 5, words))

In [4]:
random.choice(distinct_letters)

'unapt'

### Method 2: Pick words with letters that occur the most

The end goal is to eliminate as many words as possible.  For the first guess, if we pick a word that has letters that occur the most, we can see if any of those letters can be discarded.  Because we used the most frequently used letters, we theoretically eliminate more words per letter using these letters.

In [15]:
five_letters = [word for word in words if len(word) == 5]
hist = Counter()
for word in five_letters:
    for ch in word:
        hist[ch] += 1

In [20]:
hist.most_common(10)

[('a', 8393),
 ('e', 7802),
 ('s', 6537),
 ('o', 5219),
 ('r', 5144),
 ('i', 5067),
 ('l', 4247),
 ('t', 4189),
 ('n', 4044),
 ('u', 3361)]

In [30]:
possible = inc_exc(words, 'aesor', '')
possible

Words that include {'e', 's', 'o', 'a', 'r'}: 3
Words that include {'e', 's', 'o', 'a', 'r'}, but exclude set(): 3


{'arose', 'oreas', 'seora'}

## Filter words

In [21]:
five_letters = [word for word in words if len(word) == 5]
letters = defaultdict(set)
for ch in string.ascii_lowercase:
    for word in five_letters:
        if ch in word:
            letters[ch].add(word)
print(f'Words with five letters: {len(five_letters):,}')

Words with five letters: 15,920


In [34]:
possible = inc_exc(words, 'ant', 'pu')

Words that include {'a', 't', 'n'}: 302
Words that include {'a', 't', 'n'}, but exclude {'u', 'p'}: 246


In [37]:
possible = sorted(list(possible))

possible = list(filter(lambda word: word[2] == 'a', possible))
possible = list(filter(lambda word: word[1] != 'n' and word[4] != 't', possible))

print(len(possible))
possible

24


['etang',
 'neath',
 'neats',
 'niata',
 'stain',
 'stand',
 'stane',
 'stang',
 'stank',
 'starn',
 'stawn',
 'thana',
 'thane',
 'thank',
 'thatn',
 'thawn',
 'tiang',
 'train',
 'trank',
 'trans',
 'twain',
 'twana',
 'twang',
 'twank']

## Save results

In [124]:
import sqlite3

con = sqlite3.connect('wordle.db')
cur = con.cursor()

In [125]:
res = cur.execute('''SELECT * FROM results''').fetchall()
print(res)

[('319', '2022-05-04', 'unapt\ntrain', 'Wordle 319 2/6*\n\n⬜🟨🟩⬜🟨\n🟩🟩🟩🟩🟩', 'Wordle 319 2/6*\n\n⬜🟦🟧⬜🟦\n🟧🟧🟧🟧🟧')]


### Create table

Only need to do once.  Uncomment to execute.

In [119]:
# cur.execute('''
# CREATE TABLE results 
# (wordle, date, words, regular, high_contrast)
# ''')
# con.commit()

### Save result

To keep it simple, everything is stored as TEXT.

In [120]:
cur.execute('''
INSERT INTO results VALUES (
'319', '2022-05-04', 
'unapt
train', 
'Wordle 319 2/6*

⬜🟨🟩⬜🟨
🟩🟩🟩🟩🟩', 
'Wordle 319 2/6*

⬜🟦🟧⬜🟦
🟧🟧🟧🟧🟧'
)''')

# con.commit()

### SQL utilities

In [121]:
# select statements can be used as iterators or use 'fetchone' and 'fetchall' to retrieve the first matched row or a list of matched rows, respectively.
results = cur.execute('''SELECT * FROM results''').fetchall()
print(results)

[('319', '2022-05-04', 'unapt\ntrain', 'Wordle 319 2/6*\n\n⬜🟨🟩⬜🟨\n🟩🟩🟩🟩🟩', 'Wordle 319 2/6*\n\n⬜🟦🟧⬜🟦\n🟧🟧🟧🟧🟧'), ('319', '2022-05-04', 'unapt\ntrain', 'Wordle 319 2/6*\n\n⬜🟨🟩⬜🟨\n🟩🟩🟩🟩🟩', 'Wordle 319 2/6*\n\n⬜🟦🟧⬜🟦\n🟧🟧🟧🟧🟧')]


In [122]:
# cur.execute('''DELETE FROM results WHERE rowid=2''')
# con.commit()

In [123]:
# close connection to database
con.close()